In [20]:
from PIL import Image
import time
import os

def decode_binary_image(encoded_image_path, output_bin_path):
    start_time = time.time()
    img = Image.open(encoded_image_path)
    binary_message = ''
    width, height = img.size
    bit_count = None
    index = 0

    for y in range(height):
        for x in range(width):
            pixel = img.getpixel((x, y))
            for n in range(3):  # Each channel in the pixel
                if index < 32:  # Read first 32 bits for length
                    binary_message += format(pixel[n] & 0x0F, '04b')
                    index += 4
                    if index == 32:  # Once we have 32 bits, parse the length
                        bit_count = int(binary_message, 2)
                        binary_message = ''  # Reset to collect actual message bits
                else:
                    if len(binary_message) < bit_count:  # Collect only the specified number of bits
                        binary_message += format(pixel[n] & 0x0F, '04b')
                    if len(binary_message) == bit_count:  # Stop after reading bit_count bits
                        end_time = time.time()
                        decoding_time = end_time - start_time

                        # Convert the binary string to bytes and write to the output file
                        binary_data = int(binary_message, 2).to_bytes((bit_count + 7) // 8, byteorder='big')
                        with open(output_bin_path, 'wb') as file:
                            file.write(binary_data)

                        # Get the length of the output bin file in bits
                        file_size_bytes = os.path.getsize(output_bin_path)
                        file_size_bits = file_size_bytes * 8

                        return binary_message, decoding_time, file_size_bits



if __name__ == "__main__":
    encoded_image_path = 'encoded_image14.png'
    output_bin_path = 'decoded_message14.bin'
    input_bin_path = 'example.bin'

    message, decoding_time, file_size_bits = decode_binary_image(encoded_image_path, output_bin_path)
    
    print("Decoding time:", decoding_time, "seconds")
    



Decoded message: 0100100001100101011011000110110001101111001011000010000001110100011010000110100101110011001000000110100101110011001000000111001101101111011011010110010100100000011000100110100101101110011000010111001001111001001000000110010001100001011101000110000100100001
Decoding time: 0.03890824317932129 seconds
Length of output bin file: 256 bits
The binary files are identical.


In [22]:
from PIL import Image
import numpy as np

def calculate_snr(original_image_path, encoded_image_path):
    original = np.array(Image.open(original_image_path), dtype=np.float32)
    encoded = np.array(Image.open(encoded_image_path), dtype=np.float32)
    mse = np.mean((original - encoded) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    snr = 10 * np.log10(max_pixel ** 2 / mse)
    return snr

def calculate_max_capacity(width, height):
    return width * height * 3*4

def calculate_percentage_used(bits_used, max_capacity):
    return (bits_used / max_capacity) * 100

# Load images and calculate SNR
cover_image_path = 'cover_image.jpg'
encoded_image_path = 'encoded_image14.png'

snr = calculate_snr(cover_image_path, encoded_image_path)

# Image dimensions
img = Image.open(cover_image_path)
width, height = img.size

max_capacity = calculate_max_capacity(width, height)
file_size_bytes = os.path.getsize('decoded_message14.bin')
file_size_bits = file_size_bytes * 8
percentage_used = calculate_percentage_used(file_size_bits, max_capacity)

# Print results
print(f"SNR: {snr} dB")
print(f"Max Capacity: {max_capacity} bits")
print(f"Percentage Used: {percentage_used:.2f}%")

# Time complexity is O(n) for both encoding and decoding
# Memory complexity is O(1) per pixel, O(n) overall due to storing the image in memory
print("Time Complexity: O(n)")
print("Memory Complexity: O(n)")


SNR: 79.29325449604727 dB
Max Capacity: 12582912 bits
Percentage Used: 0.00%
Time Complexity: O(n)
Memory Complexity: O(n)


In [18]:
def compare_bin_files(file_path1, file_path2):
    with open(file_path1, 'rb') as file1, open(file_path2, 'rb') as file2:
        data1 = file1.read()
        data2 = file2.read()
        
    if data1 == data2:
        return True
    else:
        return False

if __name__ == "__main__":
    file1 = 'decoded_message212.bin'
    file2 = 'example.bin'

    comparison_result = compare_bin_files(file1, file2)
    if comparison_result:
        print("The binary files are identical.")
    else:
        print("The binary files are different.")


The binary files are identical.
